# LLM-based mPURIFY

## LLM-as-a-judge 

In [ ]:
#!/usr/bin/env python3
import os
import json
import time
import decimal
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from openai import AzureOpenAI
from tqdm import tqdm
import threading
import traceback
import tempfile
import shutil

 # ── GLOBAL CONFIG ──────────────────────────────────────────────────────────────
# Root of all your data files:
ROOT_DIR = r"C:\Users\jsl5710\OneDrive - The Pennsylvania State University\JasonL Research Projects\BLUFF\BLUFF_final_data"

# used to guard writes to df_meta and global_used
df_lock = threading.Lock()
cost_lock = threading.Lock()

# cache for source CSVs so we don't reload them per row
_source_csv_cache: dict[str, pd.DataFrame] = {}

# ── HELPER: extract first JSON block or fallback ───────────────────────────────
def extract_json_block(raw: str) -> str:
    # find first object or array
    idx_obj = raw.find('{')
    idx_arr = raw.find('[')
    if idx_obj == -1 and idx_arr == -1:
        return ""  # nothing JSON-like at all

    # choose earliest brace/bracket
    if 0 <= idx_arr < idx_obj or idx_obj == -1:
        start, open_c, close_c = idx_arr, '[', ']'
    else:
        start, open_c, close_c = idx_obj, '{', '}'

    depth = 0
    for i, ch in enumerate(raw[start:], start=start):
        if ch == open_c:
            depth += 1
        elif ch == close_c:
            depth -= 1
            if depth == 0:
                return raw[start:i+1]
    return ""  # malformed/unclosed


# ── RATE LIMIT CONFIG ─────────────────────────────────────────────────────────
RATE_LIMIT_PER_MINUTE = 358
MIN_INTERVAL = 60.0 / RATE_LIMIT_PER_MINUTE
_last_api_call = time.time() - MIN_INTERVAL

def _acquire_rate_limit():
    global _last_api_call
    now = time.time()
    wait = MIN_INTERVAL - (now - _last_api_call)
    if wait > 0:
        time.sleep(wait)
    _last_api_call = time.time()

# ── AZURE OPENAI CLIENT SETUP ─────────────────────────────────────────────────
client = AzureOpenAI(
    api_version="",
    azure_endpoint=" ",
    api_key=""
)

# ── COST CONFIG (OPTIONAL) ─────────────────────────────────────────────────────
COST_PER_1K_INPUT  = decimal.Decimal("0.002")
COST_PER_1K_OUTPUT = decimal.Decimal("0.008")

# ── CHARACTERISTICS MAPPING FOR FAKE NEWS ───────────────────────────────────────
characteristics = {
    "1":  "Sensational Appeal",
    "2":  "Emotionally Charged",
    "3":  "Psychologically Manipulative",
    "4":  "Misleading Statistics",
    "5":  "Fabricated Evidence",
    "6":  "Source Masking & Fake Credibility",
    "7":  "Source Obfuscation",
    "8":  "Targeted Audiences and Polarization",
    "9":  "Highly Shareable & Virality-Oriented",
    "10": "Weaponized for Political, Financial, or Social Gains",
    "11": "Simplistic, Polarizing Narratives",
    "12": "Conspiracy Framing",
    "13": "Exploits Cognitive Biases",
    "14": "Impersonation",
    "15": "Narrative Coherence Over Factual Accuracy",
    "16": "Malicious Contextual Reframing",
    "17": "False Attribution & Deceptive Endorsements",
    "18": "Exploitation of Trust in Authorities",
    "19": "Data Voids & Information Vacuum Exploitation",
    "20": "False Dichotomies & Whataboutism",
    "21": "Pseudoscience & Junk Science",
    "22": "Black Propaganda & False Flags",
    "23": "Censorship Framing & Fake Persecution",
    "24": "Astroturfing",
    "25": "Gaslighting",
    "26": "Hate Speech & Incitement",
    "27": "Information Overload & Fatigue",
    "28": "Jamming & Keyword Hijacking",
    "29": "Malinformation",
    "30": "Narrative Laundering",
    "31": "Obfuscation & Intentional Vagueness",
    "32": "Panic Mongering",
    "33": "Quoting Out of Context",
    "34": "Rumor Bombs",
    "35": "Scapegoating",
    "36": "Trolling & Provocation"
}

# ── TECHNIQUES & DEGREE FOR REAL NEWS ────────────────────────────────────────────
real_technique = {
    "2":"it rewrite text comprehensively paraphrasing and use of natural language refinement, altering wording and sentence structures",
    "1":"it polish the original content, refining clarity, flow, and readability without significantly altering the structure or factual content",
    "0":"it perform minor content editing of the original text to improve quality, correcting inaccuracies, grammar, and enhance readability"
}

# ── DEGREE MAPPINGS ────────────────────────────────────────────────────────────
fake_degree_map = {
    "minor":"Inconspicuous",
    "medium":"Moderate",
    "critical":"Alarming"
}
real_degree_map = {
    "light":    "light change (10–20%) changes",
    "moderate": "moderate change (30–50%) changes",
    "complete": "complete change (100%)"
}

# ── PROMPT TEMPLATES ───────────────────────────────────────────────────────────
FAKE_PROMPT = '''{{
  "GREEN_CoIAgenticF3": {{
    "description": "You are a master evaluator. Provided with a new article and Json generation Chains. Your task is to evaluate for consistency, change validity, translation quality, and manipulation detection based on comparison of the original text and specific chain outputs. Ensure all output is strict JSON with no extra commentary. Use single quotes in generated text to avoid parser issues.",
    "EvaluationChains": [
      {{
        "chain": "Chain [1] – Consistency Evaluator",
        "role": " You are a Evaluator/Consistency expert: Evaluate how consistent the generated content in ChainOutputs Chain [6] is with the Original Article in terms of factual correctness, logical flow, preserved meaning, tone, topic, and sentiment.",
        "guidelines": "Provide a 1–5 rating for each dimension unless otherwise noted. Match each evaluation against the Original Article versus Chain [6]. Penalize where misalignment or contradiction is introduced.",
        "evaluation": {{
          "Factual Consistency": {{
            "definition": "Accuracy of facts and details as stated in the original article.",
            "task": "Rate factual accuracy between Original Article and Chain [6]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inconsistent, partially consistent, consistent]",
            "score": "",
            "label": "_____",
            "justification": ""}},
          "Logical Consistency": {{
            "definition": "Absence of contradictions and maintenance of logical structure.",
            "task": "Rate logical alignment and absence of contradiction between Original Article and Chain [6]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree) ; label: [inconsistent, partially consistent, consistent]",
            "score": "",
            "label": "_____",
            "justification": ""}},
          "Semantic Consistency": {{
            "definition": "Preservation of key meaning and intent.",
            "task": "Rate preservation of key meaning between Original Article and Chain [6]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inconsistent, partially consistent, consistent]",
            "score": "",
            "label": "_____",
            "justification": ""}},
          "Contextual Consistency": {{
            "definition": "Alignment with the broader context and tone of the original.",
            "task": "Rate consistency of broader context and tone between Original Article and Chain [6]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inconsistent, partially consistent, consistent]",
            "score": "",
            "label": "_____",
            "justification": ""}},
          "Topic Match": {{
            "definition": "Agreement of main topic in both texts; must be concise (1–2 words only).",
            "task": "Identify and compare main topic labels from Original Article and Chain [6]. Indicate matched verdict if they match or mismatched for diverge.",
            "options": label: ["matched", "mismatched"],
            "original_topic_label": "_____ or _____  _____",
            "LLM_topic_label": "_____ or _____ _____",
            "verdict": "_____",
            "justification": ""
          }},
          "Sentiment Match": {{
            "definition": "Emotional alignment between the two versions.",
            "task": "Select sentiment polarity of Original Article and Chain [6] and compare alignment. Indicate matched verdict if they match or mismatched for diverge.",
            "options": ["positive", "neutral", "negative"],
            "original_sentiment_label": "_____",
            "LLM_sentiment_label": "_____",
            "verdict": "_____",
            "justification": ""
          }}
        }}
      }},
      {{
        "chain": "Chain [2] – Change Validity Evaluator",
        "role": "Evaluator/Change Validity Agent: Determine whether all changes logged earlier (Chain [3]) and infused tactics were correctly reflected in Chain [6] compared to the Original Article.",
        "guidelines": "All comparisons should be between the Original Article and Chain [6] with reference to changes identified in Chain [3].",
        "evaluation": {{
          "Change Validity": {{
            "definition": "Whether the documented changes were accurately applied.",
            "task": "Rate if the changes listed in chain  [3] were applied accurately and fully in Chain [6]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inaccurate, partially accurate, fully accurate]",
            "score": "",
            "label": "_____",
            "justification": ""
          }},
          "Degree of modification": {{
            "definition": "Extent of deviation from the original: Inconspicuous (minor), Moderate (medium), Alarming (critical).",
            "task": "Rate the overall extent of modification in Chain [6] relative to the Original Article",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree);label:[Inconspicuous (minor), Moderate (medium), Alarming (critical)]",
            "score": "",
            "label": "_____",
            "justification": ""
          }},
          "Edit Validation": {{
            "definition": "Contextual accuracy of individual edits.",
            "task": "Rate if individual edits in Chain [6] are contextually correct",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label:[inaccuracy, partially accuracy, fully accuracy]",
            "score": "",
            "label": "_____",
            "justification": ""
          }},
          "Technique Confirmation": {{
            "definition": "Presence of specific manipulation tactics requested by prompt including.",
            "task": "Rate whether Chain [6] includes specific rhetorical or stylistic tactics that were instructed. Also, indicate if (1) {tactic1} and (2) {tactic2} are present",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label:[one, both, none]",
            "score": "",
            "label": "",
            "justification": ""
          }}
        }}
      }},
      {{
        "chain": "Chain [3] – Translation Quality Evaluator",
        "role": "Evaluator/Translation Quality Agent: Compare the original English content in Chain [6] with its translated output in Chain [7], assessing quality and detecting correct language code.",
        "guidelines": "Base all ratings on comparison of Chain [6] (English) and Chain [7] (Translated). Penalize inaccurate or overly literal translations.",
        "evaluation": {{
          "Accurate Translation": {{
            "definition": "How precisely the meaning is retained.",
            "task": "Rate whether Chain [7] faithfully captures the meaning in Chain [6]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Fluency": {{
            "definition": "Grammatical and stylistic readability.",
            "task": "Rate fluency and readability of Chain [7]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Terminology Appropriateness": {{
            "definition": "Use of accurate domain-specific vocabulary.",
            "task": "Rate use of correct specialized terms in Chain [7]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Localization and Cultural Relevance": {{
            "definition": "Cultural sensitivity and idiomatic appropriateness.",
            "task": "Rate cultural appropriateness of Chain [7] for its language audience",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Coherence": {{
            "definition": "Logical structure and flow.",
            "task": "Rate logical structure and cohesion of Chain [7]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Semantic Quality": {{
            "definition": "Subtle meaning alignment and nuance retention.",
            "task": "Rate nuanced meaning retention in Chain [7] from Chain [6]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Language Identification": {{
            "definition": "Correct identification of language(s) from Chain [7].",
            "task": "Detect and enter ISO 639-3 code(s) for the language(s) in Chain [7]. If two or more languages are detected, return all as a list.",
            "language code": [],
            "justification": ""
          }}
        }}
      }},
      {{
        "chain": "Chain [4] – Manipulation Evaluator",
        "role": "Evaluator/Manipulation Agent: Detect signs of content manipulation, exaggeration, or distortion introduced in Chain [6] that were not present in the Original Article.",
        "guidelines": "Compare Chain [6] directly with the Original Article to flag and rate manipulation severity. Penalize if any fabricated or altered facts are inserted.",
        "evaluation": {{
          "Manipulation Detection": {{
            "definition": "Fabrication or distortion of original meaning, tone, or intent.",
            "task": "Rate manipulation between Original Article and Chain [6]",
            "options": "(1 = No Manipulation, 5 = Extensive Manipulation); label: [no manipulation, partial manipulation, full manipulation]",
            "score": "",
            "label": "_____",
            "patterns_found": [],
            "justification": ""
          }}
        }}
      }}
    ],
    "Original Article": "{article}",
    "ChainOutputs": "{json_chain}"
  }}
}}'''

REAL_PROMPT = '''{{
  "GREEN_CoIAgenticF3": {{
    "description": "You are a master evaluator. Provided with a new article and Json generation Chains. Your task is to evaluate for consistency, change validity, translation quality, and manipulation detection based on comparison of the original text and specific chain outputs. Ensure all output is strict JSON with no extra commentary. Use single quotes in generated text to avoid parser issues.",
    "EvaluationChains": [
      {{
        "chain": "Chain [1] – Consistency Evaluator",
        "role": " You are a Evaluator/Consistency expert: Evaluate how consistent the generated content in ChainOutputs Chain [4] is with the Original Article in terms of factual correctness, logical flow, preserved meaning, tone, topic, and sentiment.",
        "guidelines": "Provide a 1–5 rating for each dimension unless otherwise noted. Match each evaluation against the Original Article versus Chain [4]. Penalize where misalignment or contradiction is introduced.",
        "evaluation": {{
          "Factual Consistency": {{
            "definition": "Accuracy of facts and details as stated in the original article.",
            "task": "Rate factual accuracy between Original Article and Chain [4]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inconsistent, partially consistent, consistent]",
            "score": "",
            "label": "_____",
            "justification": ""}},
          "Logical Consistency": {{
            "definition": "Absence of contradictions and maintenance of logical structure.",
            "task": "Rate logical alignment and absence of contradiction between Original Article and Chain [4]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inconsistent, partially consistent, consistent]",
            "score": "",
            "label": "_____",
            "justification": ""}},
          "Semantic Consistency": {{
            "definition": "Preservation of key meaning and intent.",
            "task": "Rate preservation of key meaning between Original Article and Chain [4]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inconsistent, partially consistent, consistent]",
            "score": "",
            "label": "_____",
            "justification": ""}},
          "Contextual Consistency": {{
            "definition": "Alignment with the broader context and tone of the original.",
            "task": "Rate consistency of broader context and tone between Original Article and Chain [4]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inconsistent, partially consistent, consistent]",
            "score": "",
            "label": "_____",
            "justification": ""}},
          "Topic Match": {{
            "definition": "Agreement of main topic in both texts; must be concise (1–2 words only).",
            "task": "Identify and compare main topic labels from Original Article and Chain [4]. Indicate matched verdict if they match or mismatched for diverge.",
            "options": label: ["matched", "mismatched"],
            "original_topic_label": "_____ or _____  _____",
            "LLM_topic_label": "_____ or _____ _____",
            "verdict": "_____",
            "justification": ""
          }},
          "Sentiment Match": {{
            "definition": "Emotional alignment between the two versions.",
            "task": "Select sentiment polarity of Original Article and Chain [4] and compare alignment. Indicate matched verdict if they match or mismatched for diverge.",
            "options": ["positive", "neutral", "negative"],
            "original_sentiment_label": "_____",
            "LLM_sentiment_label": "_____",
            "verdict": "_____",
            "justification": ""
          }}
        }}
      }},
      {{
        "chain": "Chain [2] – Change Validity Evaluator",
        "role": "Evaluator/Change Validity Agent: Determine whether all changes logged earlier (Chain [3]) and infused tactics were correctly reflected in Chain [4] compared to the Original Article.",
        "guidelines": "All comparisons should be between the Original Article and Chain [4] with reference to changes identified in Chain [3].",
        "evaluation": {{
          "Change Validity": {{
            "definition": "Whether the documented changes were accurately applied.",
            "task": "Rate if the changes listed in chain  [3] were applied accurately and fully in Chain [4]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Degree of modification": {{
            "definition": "Extent of deviation from the original real news version.",
            "task": "Rate the overall extent of modification in Chain [4] relative to the Original Article",
            "options": "label: [light = light change (10–20%), moderate = moderate change (30–50%), complete = complete change (100%)]",
            "score": "",
            "label": "_____",
            "justification": ""
          }},
          "Edit Validation": {{
            "definition": "Contextual accuracy of individual edits.",
            "task": "Rate if individual edits in Chain [4] are contextually correct",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [inaccuracy, partially accuracy, fully accuracy]",
            "score": "",
            "label": "_____",
            "justification": ""
          }},
          "Technique Confirmation": {{
            "definition": "Presence of specific editing techniques as instructed (e.g., polish, rewrite, simplify).",
            "task": "Rate whether Chain [4] includes the requested real news editing technique. Also, indicate if (1) {tech} was done.",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree); label: [not-done, partially done, fully done]",
            "score": "",
            "label": "_____",
            "justification": ""
          }}
        }}
      }},
      {{
        "chain": "Chain [3] – Translation Quality Evaluator",
        "role": "Evaluator/Translation Quality Agent: Compare the original English content in Chain [4] with its translated output in Chain [5], assessing quality and detecting correct language code.",
        "guidelines": "Base all ratings on comparison of Chain [4] (English) and Chain [5] (Translated). Penalize inaccurate or overly literal translations.",
        "evaluation": {{
          "Accurate Translation": {{
            "definition": "How precisely the meaning is retained.",
            "task": "Rate whether Chain [5] faithfully captures the meaning in Chain [4]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Fluency": {{
            "definition": "Grammatical and stylistic readability.",
            "task": "Rate fluency and readability of Chain [5]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Terminology Appropriateness": {{
            "definition": "Use of accurate domain-specific vocabulary.",
            "task": "Rate use of correct specialized terms in Chain [5]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Localization and Cultural Relevance": {{
            "definition": "Cultural sensitivity and idiomatic appropriateness.",
            "task": "Rate cultural appropriateness of Chain [5] for its language audience",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Coherence": {{
            "definition": "Logical structure and flow.",
            "task": "Rate logical structure and cohesion of Chain [5]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Semantic Quality": {{
            "definition": "Subtle meaning alignment and nuance retention.",
            "task": "Rate nuanced meaning retention in Chain [5] from Chain [4]",
            "options": "(1 = Strongly Disagree, 5 = Strongly Agree)",
            "score": "",
            "justification": ""
          }},
          "Language Identification": {{
            "definition": "Correct identification of language(s) from Chain [5].",
            "task": "Detect and enter ISO 639-3 code(s) for the language(s) in Chain [5]. If two or more languages are detected, return all as a list.",
            "language code": [],
            "justification": ""
          }}
        }}
      }},
      {{
        "chain": "Chain [4] – Manipulation Evaluator",
        "role": "Evaluator/Manipulation Agent: Detect signs of content manipulation, exaggeration, or distortion introduced in Chain [4] that were not present in the Original Article.",
        "guidelines": "Compare Chain [4] directly with the Original Article to flag and rate manipulation severity. Penalize if any fabricated or altered facts are inserted.",
        "evaluation": {{
          "Manipulation Detection": {{
            "definition": "Fabrication or distortion of original meaning, tone, or intent.",
            "task": "Rate manipulation between Original Article and Chain [4]",
            "options": "(1 = No Manipulation, 5 = Extensive Manipulation); label: [no manipulation, partial manipulation, full manipulation]",
            "score": "",
            "label": "_____",
            "patterns_found": [],
            "justification": ""
          }}
        }}
      }}
    ],
    "Original Article": "{article}",
    "ChainOutputs": "{json_chain}"
  }}
}}'''

# ── PROMPT BUILDER ─────────────────────────────────────────────────────────────
def build_prompt(article, json_chain, veracity, tactics, degree_key):
    # Option 2: strip brackets before splitting
    # join in case tactics is like ["[8", " 19]"]
    raw = ",".join(tactics).strip("[] ")
    clean_tactics = [t.strip() for t in raw.split(",") if t.strip()]

    if veracity == 'fake_news':
        if len(clean_tactics) < 2:
            raise ValueError(f"fake_news requires 2 tactics, got: {clean_tactics!r}")
        key1, key2 = clean_tactics[0], clean_tactics[1]

        tac1 = characteristics.get(key1, "tactic 1")
        tac2 = characteristics.get(key2, "tactic 2")

        return FAKE_PROMPT.format(
            article=article,
            json_chain=json_chain,
            tactic1=tac1,
            tactic2=tac2,
        )

    elif veracity == 'real_news':
        if not clean_tactics:
            raise ValueError(f"real_news requires at least 1 technique, got: {clean_tactics!r}")
        tech_key = clean_tactics[0]

        tech_desc = real_technique.get(tech_key, "text modification")

        return REAL_PROMPT.format(
            article=article,
            json_chain=json_chain,
            tech=tech_desc,
        )

    else:
        raise ValueError(f"Unknown veracity: {veracity!r}")


# ── MAIN OUTPUT & STATE TRACKING ───────────────────────────────────────────────
main_output_dir = "./evaluations"
os.makedirs(main_output_dir, exist_ok=True)
used_file = os.path.join(main_output_dir, "global_used_uuids.json")
if os.path.exists(used_file):
    with open(used_file, 'r', encoding='utf-8') as f:
        global_used = set(json.load(f))
else:
    global_used = set()
    with open(used_file, 'w', encoding='utf-8') as f:
        json.dump([], f)

# ── LOAD METADATA ──────────────────────────────────────────────────────────────
metadata_path = "C:\\Users\\jsl5710\OneDrive - The Pennsylvania State University\JasonL Research Projects\\BLUFF\\BLUFF_final_data\\metadata_final_clean.csv"
metadata_with_eval_path = "C:\\Users\\jsl5710\OneDrive - The Pennsylvania State University\JasonL Research Projects\\BLUFF\\BLUFF_final_data\\metadata_final_clean_x_LLM_mPURIFY_eval.csv"

df_meta = pd.read_csv(metadata_path, dtype={'uuid': str})
if 'evaluation_path' not in df_meta.columns:
    df_meta['evaluation_path'] = None

# ── PROCESS & SAVE ─────────────────────────────────────────────────────────────
def save_output(uuid, eval_type, content):
    subdir = os.path.join(main_output_dir, f"{eval_type}_eval")
    os.makedirs(subdir, exist_ok=True)
    path = os.path.join(subdir, f"{uuid}_{eval_type}eval.json")
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(content, f, indent=2, ensure_ascii=False)
    return path

# ── IMPROVED COST LEDGER WITH ATOMIC WRITES ──
cost_file = os.path.join(main_output_dir, "overall_costs.json")

def save_cost_record(record: dict):
    """
    Append a single call's cost record with atomic file operations
    to prevent JSON corruption from concurrent access.
    """
    with cost_lock:
        max_retries = 3
        for attempt in range(max_retries):
            try:
                # Try to load existing data
                if os.path.exists(cost_file):
                    with open(cost_file, "r", encoding="utf-8") as f:
                        data = json.load(f)
                else:
                    data = {"total_cost": 0.0, "calls": []}

                # Add new record
                data["calls"].append(record)
                data["total_cost"] += record.get("cost", 0.0)

                # Write atomically using temporary file
                with tempfile.NamedTemporaryFile(mode='w', delete=False, 
                                               dir=main_output_dir, 
                                               suffix='.tmp') as tmp_f:
                    json.dump(data, tmp_f, indent=2, ensure_ascii=False)
                    tmp_name = tmp_f.name

                # Atomic move
                shutil.move(tmp_name, cost_file)
                break  # Success, exit retry loop

            except (json.JSONDecodeError, IOError) as e:
                print(f"[WARN] Cost file error (attempt {attempt + 1}/{max_retries}): {e}")
                if attempt == max_retries - 1:
                    print(f"[ERROR] Failed to save cost record after {max_retries} attempts")
                    # Create backup record
                    backup_file = os.path.join(main_output_dir, f"cost_backup_{int(time.time())}.json")
                    with open(backup_file, 'w', encoding='utf-8') as f:
                        json.dump([record], f, indent=2)
                else:
                    time.sleep(0.1)  # Brief delay before retry


def load_source_csv_safe(src_csv_path):
    """Thread-safe CSV loading with caching"""
    with df_lock:
        if src_csv_path not in _source_csv_cache:
            try:
                _source_csv_cache[src_csv_path] = pd.read_csv(src_csv_path, dtype={'uuid': str})
            except Exception as e:
                print(f"[ERROR] Failed to load CSV {src_csv_path}: {e}")
                return None
        return _source_csv_cache[src_csv_path]

# ── PROCESS EACH ROW WITH BETTER ERROR HANDLING ───────────────────────────────────────────────────────────
def process_row(idx, row):
    global df_meta, global_used

    uuid = str(row['uuid'])
    if uuid in global_used:
        return

    try:
        # Load article from source CSV with thread safety
        src_rel = row['source_dir'].lstrip("/\\")
        src_csv_path = os.path.join(ROOT_DIR, src_rel)
        df_src = load_source_csv_safe(src_csv_path)
        if df_src is None:
            return

        try:
            article = (
                df_src.loc[df_src['uuid'] == uuid, 'content']
                      .iloc[0]
                      .replace('"', '\\"')
            )
        except IndexError:
            print(f"[ERROR] UUID {uuid} not found in source CSV {src_csv_path}")
            return

        # Load JSON chain with better error handling
        json_rel = row['json_dir'].lstrip("/\\")
        json_path = os.path.join(ROOT_DIR, json_rel)
        try:
            with open(json_path, 'r', encoding='utf-8-sig') as f:
                raw = f.read()
        except FileNotFoundError:
            print(f"[ERROR] JSON file not found at {json_path}")
            return
        except Exception as e:
            print(f"[ERROR] Failed to read JSON file {json_path}: {e}")
            return

        snippet = extract_json_block(raw)
        if snippet:
            try:
                obj = json.loads(snippet)
                json_chain = json.dumps(obj).replace('"', '\\"')
            except json.JSONDecodeError:
                print(f"[WARN] Extracted JSON malformed in {json_path}; using raw snippet.")
                json_chain = snippet.replace('"', '\\"')
        else:
            print(f"[WARN] No JSON block in {json_path}; loading full content as text.")
            json_chain = raw.replace('"', '\\"')

        # Build prompt with error handling
        tactics = [t.strip() for t in str(row['technique_keys']).split(',')]
        degree_key = row['degree']
        veracity = row['veracity']
        
        try:
            prompt = build_prompt(article, json_chain, veracity, tactics, degree_key)
        except Exception as e:
            print(f"[ERROR] Failed to build prompt for {uuid}: {e}")
            return

        messages = [
            {"role": "system", "content": "You are a master evaluator. Return strict JSON only."},
            {"role": "user", "content": prompt}
        ]

        # API call with rate limiting and error handling
        _acquire_rate_limit()
        try:
            resp = client.chat.completions.create(model="gpt-4.1", messages=messages)
            
            # Track cost
            try:
                pt = resp.usage.prompt_tokens
                ct = resp.usage.completion_tokens
                cost = float(
                    (decimal.Decimal(pt) / 1000) * COST_PER_1K_INPUT +
                    (decimal.Decimal(ct) / 1000) * COST_PER_1K_OUTPUT
                )
                save_cost_record({
                    "uuid": uuid,
                    "prompt_tokens": pt,
                    "completion_tokens": ct,
                    "cost": cost,
                    "timestamp": time.time()
                })
            except AttributeError:
                print(f"[WARN] API response missing usage data for {uuid}")

        except Exception as e:
            print(f"[ERROR] API call failed for {uuid}: {e!r}")
            return

        # Parse response
        try:
            parsed = json.loads(resp.choices[0].message.content)
        except json.JSONDecodeError:
            print(f"[WARN] Response not valid JSON for {uuid}, storing raw content")
            parsed = resp.choices[0].message.content

        # Save output and update metadata
        try:
            out_path = save_output(uuid, veracity, parsed)
            with df_lock:
                df_meta.at[idx, 'evaluation_path'] = out_path
                global_used.add(uuid)
                
                # Save used UUIDs atomically
                with tempfile.NamedTemporaryFile(mode='w', delete=False, 
                                               dir=main_output_dir, suffix='.tmp') as tmp_f:
                    json.dump(list(global_used), tmp_f)
                    tmp_name = tmp_f.name
                shutil.move(tmp_name, used_file)
                
                # Save metadata
                df_meta.to_csv(metadata_with_eval_path, index=False)
                
        except Exception as e:
            print(f"[ERROR] Failed to save output for {uuid}: {e!r}")

    except Exception as e:
        print(f"[ERROR] Unexpected error processing {uuid}: {e!r}")
        traceback.print_exc()

# ── EXECUTION WITH REDUCED THREAD COUNT ─────────────────────────────────────────────────────────────────
if __name__ == '__main__':
    total = len(df_meta)
    # Reduced thread count to minimize contention
    with ThreadPoolExecutor(max_workers=20) as executor:  # Reduced from 60
        futures = [executor.submit(process_row, idx, row)
                   for idx, row in df_meta.iterrows()]
        for _ in tqdm(as_completed(futures), total=total, desc="Evaluating News"):
            pass

    print(f"All evaluations complete. Latest metadata at {metadata_with_eval_path}")

C:\Users\jsl5710\AppData\Local\Temp\ipykernel_2044\1498442487.py:535: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv(metadata_path, dtype={'uuid': str})
Evaluating News:  89%|████████▊ | 59842/67531 [00:57<03:34, 35.79it/s]  

[WARN] Response not valid JSON for af330011-6fb3-4a61-ad0c-9855deb2004b, storing raw content


Evaluating News:  90%|████████▉ | 60678/67531 [02:17<26:25,  4.32it/s]

[WARN] Response not valid JSON for bad7923a-9dd9-46d5-8a2f-70e4f9f7423e, storing raw content


Evaluating News:  91%|█████████ | 61229/67531 [03:04<15:38,  6.71it/s]

...

[WARN] Response not valid JSON for 94dfae14-783e-4128-98cf-a8b376fa784f, storing raw content


Evaluating News: 100%|█████████▉| 67432/67531 [2:15:01<02:45,  1.67s/it]

[WARN] Response not valid JSON for f32cea45-4e06-42a7-afe0-ad0f19793a39, storing raw content


Evaluating News: 100%|██████████| 67531/67531 [2:17:41<00:00,  8.17it/s]

All evaluations complete. Latest metadata at C:\Users\jsl5710\OneDrive - The Pennsylvania State University\JasonL Research Projects\BLUFF\BLUFF_final_data\metadata_final_clean_x_LLM_mPURIFY_eval.csv
